# Model 1: MultiNomial Naive Bayes

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

## Loading the Dataset

In [57]:
data = pd.read_csv('cleaned_news_data.csv')

data.info()
print()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1549 entries, 0 to 1548
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1549 non-null   int64 
 1   title       1549 non-null   object
 2   link        1549 non-null   object
 3   content     1549 non-null   object
 4   gold_label  1549 non-null   object
dtypes: int64(1), object(4)
memory usage: 60.6+ KB



,id,title,link,content,gold_label
0,0,کبھی میں کبھی تم‘ کا محبت بھرا اختتام، شرجینا ...,https://www.express.pk/story/2731764/kbhi-main...,کبھی میں کبھی تم کی آخری قسط میں شرجینا اور مص...,Entertainment
1,1,پاکستانی اداکار احسن خان کی کرس گیل کے ساتھ ڈا...,https://www.express.pk/story/2731730/ahsan-kha...,پاکستانی اداکار احسن خان کی ویسٹ انڈیز کے سابق...,Entertainment
2,2,اداکارہ لائبہ خان نے عمرے کی سعادت حاصل کرلی,https://www.express.pk/story/2731719/pakistani...,پاکستانی اداکارہ لائبہ خان نے اپنی فیملی کے ہم...,Entertainment
3,3,ویرات کوہلی کی 36 ویں سالگرہ پر انوشکا نے ’اکے...,https://www.express.pk/story/2731713/anushka-s...,انوشکا شرما نے اپنے شوہر ویرات کوہلی کی 36 ویں...,Entertainment
4,4,شاہ رخ خان نے 95 دن سے گھر کے باہر بیٹھے مداح ...,https://www.express.pk/story/2731697/shah-rukh...,بالی ووڈ اسٹار شاہ رخ خان نے 95 دن سے گھر کے ب...,Entertainment


## Preprocessing the Dataset

In [58]:

with open('stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

def preprocessor(text):
    
    text = ' '.join([word for word in text.split() if word not in stopwords])
    
    return text

data['title'] = data['title'].apply(preprocessor)
data['content'] = data['content'].apply(preprocessor)

data['combined'] = data['title'] + " " + data['content']

data.head()


,id,title,link,content,gold_label,combined
0,0,کبھی میں کبھی تم‘ کا محبت بھرا اختتام، شرجینا ...,https://www.express.pk/story/2731764/kbhi-main...,کبھی میں کبھی تم آخری قسط میں شرجینا مصطفیٰ نے...,Entertainment,کبھی میں کبھی تم‘ کا محبت بھرا اختتام، شرجینا ...
1,1,پاکستانی اداکار احسن خان کرس گیل ساتھ ڈانس ویڈ...,https://www.express.pk/story/2731730/ahsan-kha...,پاکستانی اداکار احسن خان ویسٹ انڈیز سابق کرکٹر...,Entertainment,پاکستانی اداکار احسن خان کرس گیل ساتھ ڈانس ویڈ...
2,2,اداکارہ لائبہ خان نے عمرے سعادت حاصل کرلی,https://www.express.pk/story/2731719/pakistani...,پاکستانی اداکارہ لائبہ خان نے اپنی فیملی ہمراہ...,Entertainment,اداکارہ لائبہ خان نے عمرے سعادت حاصل کرلی پاکس...
3,3,ویرات کوہلی 36 ویں سالگرہ انوشکا نے ’اکے‘ جھلک...,https://www.express.pk/story/2731713/anushka-s...,انوشکا شرما نے اپنے شوہر ویرات کوہلی 36 ویں سا...,Entertainment,ویرات کوہلی 36 ویں سالگرہ انوشکا نے ’اکے‘ جھلک...
4,4,شاہ رخ خان نے 95 دن سے گھر باہر بیٹھے مداح کو ...,https://www.express.pk/story/2731697/shah-rukh...,بالی ووڈ اسٹار شاہ رخ خان نے 95 دن سے گھر باہر...,Entertainment,شاہ رخ خان نے 95 دن سے گھر باہر بیٹھے مداح کو ...


In [59]:
X = data['content'] 
y = data['gold_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



(1239,) (1239,)
(310,) (310,)


## Implementation of Multinomial Naive Bayes From Scratch

In [60]:
class BagOfWords:
    def __init__(self):
        self.vocabulary = {}  
        self.vocab_size = 0

    def fit(self, corpus):

        unique_words = set()

        for sentence in corpus:
            words = sentence.split() 
            unique_words.update(words)  

        self.vocabulary = {word: idx for idx, word in enumerate(sorted(unique_words))}
        self.vocab_size = len(self.vocabulary)

    def vectorize(self, sentence):

        vector = [0] * self.vocab_size
        
        words = sentence.split()

        for word in words:
            if word in self.vocabulary:
                index = self.vocabulary[word]
                vector[index] += 1

        return vector

In [61]:
model = BagOfWords()
model.fit(X_train)

vector_training = [model.vectorize(doc) for doc in X_train]
vector_test = [model.vectorize(doc) for doc in X_test]


In [62]:
class MultiNaiveBayes:
    def __init__(self):
        self.classprobs = {}  
        self.wordprobs = {}  
        self.vocab_size = 0    
        self.bow = BagOfWords()  # Assume BagOfWords handles vectorization without LabelEncoder                

    def fit(self, X, y):
        self.bow.fit(X)         
        X_vectorized = [self.bow.vectorize(doc) for doc in X] 
        self.vocab_size = self.bow.vocab_size 
        n_docs = len(X)
        
        # Get unique classes as they are
        unique_classes = np.unique(y)  
        class_counts = {c: 0 for c in unique_classes}
        word_counts = {c: np.zeros(self.vocab_size) for c in unique_classes}

        for i in range(n_docs):
            c = y.iloc[i]  # Updated to y.iloc[i] to handle pandas Series correctly
            class_counts[c] += 1
            word_counts[c] += X_vectorized[i]

        for c in unique_classes:
            self.classprobs[c] = class_counts[c] / n_docs
            total_words_in_class = np.sum(word_counts[c])
            self.wordprobs[c] = (word_counts[c] + 1) / (total_words_in_class + self.vocab_size)
    
    def predict(self, X):
        X_vectorized = np.array([self.bow.vectorize(doc) for doc in X])
        predictions = []
        
        for doc_vec in X_vectorized:
            class_scores = {}
            for c in self.classprobs:
                log_prob_c = np.log(self.classprobs[c])
                log_prob_x_given_c = np.sum(doc_vec * np.log(self.wordprobs[c]) + (1 - doc_vec) * np.log(1 - self.wordprobs[c]))
                class_scores[c] = log_prob_c + log_prob_x_given_c
            
            best_class = max(class_scores, key=class_scores.get)
            predictions.append(best_class)
        
        return predictions


In [63]:
model = MultiNaiveBayes()
model.fit(X_train, y_train) 

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
confusionmatrix = confusion_matrix(y_test, predictions)

print("Manually Implemented Naive Bayes:")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print(confusionmatrix)

Manually Implemented Naive Bayes:
Accuracy:  0.8516129032258064
Precision:  0.8688089427219863
Recall:  0.8530807139542249
F1 Score:  0.8468886281752823
[[42  1  1  0  0]
 [11 73  1  1  0]
 [ 9  0 61  0  0]
 [ 3  1  2 22  0]
 [ 9  3  4  0 66]]


In [64]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(vector_training, y_train)
mnbprediction = model.predict(vector_test)

accuracy = accuracy_score(y_test, mnbprediction)
precision = precision_score(y_test, mnbprediction, average='macro')
recall = recall_score(y_test, mnbprediction, average='macro')
f1 = f1_score(y_test, mnbprediction, average='macro')
confusionmatrix = confusion_matrix(y_test, mnbprediction)

print("For Sklearn MultiNomial Naive Bayes:")
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print(confusionmatrix)
print()

For Sklearn MultiNomial Naive Bayes:
Accuracy:  0.8516129032258064
Precision:  0.8688089427219863
Recall:  0.8530807139542249
F1 Score:  0.8468886281752823
[[42  1  1  0  0]
 [11 73  1  1  0]
 [ 9  0 61  0  0]
 [ 3  1  2 22  0]
 [ 9  3  4  0 66]]

